<a href="https://colab.research.google.com/github/sophiezla/FII-NX1Intl-SOPHIE-ZHU/blob/main/counterfeit_drug_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Counterfeit drugs and flaseified medicines create a huge risk towards public health. Drug counterfeiting has increased greatly in recent years and have made their way into the legal markets.
They can be extremely harmful if taken- from contamination to under/over dosage which makes identifying and eliminating these drugs so important.
https://www.clientpharma.com/the-importance-of-detecting-counterfeit-medicines/

A very good resource right now : https://www.whpa.org/sites/default/files/2018-12/Toolkit_BeAware_Background.pdf